<a href="https://colab.research.google.com/github/nazarb/Mining/blob/main/Yolov5_test6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
logging.getLogger().setLevel(logging.CRITICAL)
import os 
import zipfile

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

fid = drive.ListFile({'q':"title='Wes_11_3_wgs_YOLO_v2.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('Wes_11_3_wgs_YOLO_v2.zip')

In [ ]:

import zipfile
with zipfile.ZipFile("/content/Wes_11_3_wgs_YOLO_v2.zip", 'r') as zip_ref:
    zip_ref.extractall("./")

In [ ]:
import shutil
shutil.move("/content/WeS_11_3_wgs_YOLO_v2/Dataset", "/content/Dataset")

%rm -rf "Wes_11_3_wgs_YOLO_v2"

In [ ]:
!cat /content/Dataset/dataset.yaml

names:
- other
- mine

nc: 
- 0
- 1

path: ..
test: /content/Dataset/images/test
train: /content/Dataset/images/train
val: /content/Dataset/images/val

In [ ]:
# set up environment
os.environ["DATASET_DIRECTORY"] = "./content/Dataset/"

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 15305, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 15305 (delta 0), reused 4 (delta 0), pack-reused 15300
Receiving objects: 100% (15305/15305), 14.19 MiB | 23.03 MiB/s, done.
Resolving deltas: 100% (10493/10493), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.9.0 requires jedi>=0.10, which is not installed.
cvxpy 1.2.3 requires setuptools<=64.0.2, but you have setuptools 67.6.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 KB 3.2 MB/s eta 0:

In [ ]:
!python train.py --img 512 --batch 16 --epochs 150 --data /content/Dataset/dataset.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=/content/Dataset/dataset.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=512, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_

In [ ]:
# Validate YOLOv5s on COCO val
!python val.py --weights /content/yolov5/runs/train/exp/weights/best.pt --data /content/Dataset/dataset.yaml --img 512 --half

val: data=/content/Dataset/dataset.yaml, weights=['/content/yolov5/runs/train/exp/weights/best.pt'], batch_size=32, imgsz=512, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=True, dnn=False
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
val: Scanning /content/Dataset/labels/val.cache... 100 images, 82 backgrounds, 0 corrupt: 100% 100/100 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 4/4 [00:02<00:00,  1.44it/s]
                   all        100         92      0.634      0.457      0.512      0.253
                  mine        100         92      0.634      0.457      0.512      0.253
Speed: 1.1ms p

In [ ]:
from IPython.display import Image, display


!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 512 --conf 0.2 --source /content/Dataset/images/test/

display(filename='../content/Dataset/images/test/209.tif', width=512)


detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/Dataset/images/test/, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/100 /content/Dataset/images/test/1.tif: 512x512 (no detections), 11.1ms
image 2/100 /content/Dataset/images/test/10.tif: 512x512 (no detections), 10.1ms
image 3/100 /content/Dataset/images/test/100.tif: 512x512 (no detections), 10.4ms
image 4/100 /content/Dataset/images/test/101.tif: 512x512 (no detections), 10.

In [ ]:
!zip -r WeS_11_3_wgs_YOLO_results_2_02.zip /content/yolov5/runs

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/val/ (stored 0%)
  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 22%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 38%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 15%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:

import shutil

shutil.copy("/content/yolov5/WeS_11_3_wgs_YOLO_results_2_02.zip","/content/drive/MyDrive/Mining")

'/content/drive/MyDrive/Mining/WeS_11_3_wgs_YOLO_results_2_02.zip'

Detect Other dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


shutil.copy("/content/drive/MyDrive/Mining/WeS_22_2_wgs_YOLO_validation.zip","/content/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


'/content/WeS_22_2_wgs_YOLO_validation.zip'

In [ ]:

import zipfile
with zipfile.ZipFile("/content/WeS_22_2_wgs_YOLO_validation.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/")

In [ ]:
os.rename("/content/content/Dataset","/content/content/Validation")

FileNotFoundError: ignored

In [ ]:
import shutil
shutil.move("/content/content/Validation", "/content/Validation")

%rm -rf "/content/content/"

FileNotFoundError: ignored

In [ ]:
from IPython.display import Image, display


!python detect.py --weights /content/yolov5/runs/train/exp/weights/best.pt --img 512 --conf 0.2 --source /content/Validation/images/test

display(filename='/content/Validation/images/test/1.tif', width=512)

detect: weights=['/content/yolov5/runs/train/exp/weights/best.pt'], source=/content/Validation/images/test, data=data/coco128.yaml, imgsz=[512, 512], conf_thres=0.2, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-120-g3e55763 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 157 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
image 1/100 /content/Validation/images/test/0.tif: 512x512 (no detections), 8.9ms
image 2/100 /content/Validation/images/test/1.tif: 512x512 10 mines, 8.9ms
image 3/100 /content/Validation/images/test/10.tif: 512x512 (no detections), 9.0ms
image 4/100 /content/Validation/images/test/11.tif: 512x512 (no detections), 9.

In [ ]:
!zip -r WeS_11_3_wgs_YOLO_results_val_WES_22_2.zip /content/yolov5/runs

  adding: content/yolov5/runs/ (stored 0%)
  adding: content/yolov5/runs/val/ (stored 0%)
  adding: content/yolov5/runs/val/exp/ (stored 0%)
  adding: content/yolov5/runs/val/exp/val_batch0_labels.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/PR_curve.png (deflated 22%)
  adding: content/yolov5/runs/val/exp/R_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch0_pred.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/F1_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch1_labels.jpg (deflated 15%)
  adding: content/yolov5/runs/val/exp/confusion_matrix.png (deflated 38%)
  adding: content/yolov5/runs/val/exp/val_batch2_pred.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/P_curve.png (deflated 17%)
  adding: content/yolov5/runs/val/exp/val_batch2_labels.jpg (deflated 14%)
  adding: content/yolov5/runs/val/exp/val_batch1_pred.jpg (deflated 15%)
  adding: content/yolov5/runs/train/ (stored 0%)
  adding: content/yolov5/runs/

In [ ]:
import shutil

shutil.copy("/content/yolov5/WeS_11_3_wgs_YOLO_results_val_WES_22_2.zip","/content/drive/MyDrive/Mining")

'/content/drive/MyDrive/Mining/WeS_11_3_wgs_YOLO_results_val_WES_22_2.zip'